In [4]:
import os
from glob import glob
import shutil
from pathlib import Path, PurePath

import json

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import cv2

from sklearn.model_selection import train_test_split

import torchvision
from torchvision.transforms import transforms as T
from torchvision.utils import draw_bounding_boxes

from tqdm import tqdm

import albumentations as A

import time

%matplotlib inline
torch.manual_seed(42)


ModuleNotFoundError: No module named 'albumentations'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device